## CPEX-AW Dropsonde Plotting Code (Questions?  Email brodenkirch@wisc.edu)

#### Given a filepath (dropsonde_folder, see cell #3) that contains CPEX-AW dropsonde ".nc" (NetCDF) files, this code will:

1. Loop through each dropsonde file to filter/QC the data and add all filtered/QCed dropsonde data to the given date's (file_date, see cell #2) created final_dropsonde CSV, saved to day_folder. (cell #4)

2. Make "height vs. time" dropsonde availability plots for the given file_date, saved to day_folder. (cell #5)

3. Make theta, theta-e, and theta-v plots for each dropsonde profile, saved to day_folder. (cell #6)

4. Make skew-T plots (not publication quality) for each dropsonde profile using SHARPpy, saved to day_folder. (cells #7 and #8)

5. Calculate mean-layer metrics (e.g., RH, wind shear, etc.) for each dropsonde using SHARPpy. (cell #9)  These metrics are not labeled in the output, but are printed in the order of the variables below **(NOTE:  some of these metrics purposefully match up with blank values, mostly because these blank metrics were meant to be manually inputted and/or were not useful/relevant anymore).**  The mid-layer is defined as extending from the PBL top up to the freezing level.  The upper-layer is defined as extending from the freezing level up to the lowmax_hght (**you probably will want to change this!**).  The deep-layer is defined as extending from the near-surface (the given dropsonde's lowest available height level) up to the lowmax_hght.

* Sfc Pressure [mb]
* Sfc Height [m]
* PBL Top [mb] 
* PBL Top Height [m] 
* Freezing Level [mb] 
* Freezing Level Height [m]
* Upper Level Cap [mb] 
* Upper Level Cap Height [m]
* M10 Celsius Level [mb] 
* M20 Celsius Level [mb]
* M30 Celsius Level [mb]
* PBL RH [%]
* Mid Layer RH [%] 
* Upper Layer RH [%]
* Deep Layer RH [%]
* Below FZL MUCAPE [J/kg] 
* Above FZL MUCAPE [J/kg] 
* Deep Layer MUCAPE [J/kg] 
* MU CIN [J/kg]
* MU LCL [mb]
* MU EL [mb] 
* MU Max LI [degC] 
* MU Max LI Level [mb] 
* MU Max LI Layer (always blank)
* Below FZL MLCAPE [J/kg]
* Above FZL MLCAPE [J/kg]	
* Deep Layer MLCAPE [J/kg]
* ML CIN [J/kg]
* ML LCL [mb]
* ML EL [mb] 
* ML Max LI [degC]
* ML Max LI Level [mb] 
* ML Max LI Layer (always blank)
* SHARPpy Direct Method PBL Speed Shear [kts]	
* SHARPpy Direct Method PBL Directional Shear [deg] 
* SHARPpy Direct Method Mid Layer Speed Shear [kts] 
* SHARPpy Direct Method Mid Layer Directional Shear [deg] 
* SHARPpy Direct Method Upper Layer Speed Shear [kts]	
* SHARPpy Direct Method Upper Layer Directional Shear [deg]
* SHARPpy Direct Method Deep Layer Speed Shear [kts] 
* SHARPpy Direct Method Deep Layer Directional Shear [deg] 
* 10mb Interval Theta Gradient PBL Top [mb] (always blank)
* 10mb Interval Theta Gradient PBL RH [%] (always blank)
* 10mb Interval Theta Gradient Mid Layer RH [%] (always blank)
* Max Profile Height [m]
* 500m Bottom Cap Profile Height [m] 
* 500m Bottom Cap Deep Layer Speed Shear [kts] 
* 500m Bottom Cap Deep Layer Directional Shear [deg]

### Hope this helps!  Feel free to edit the code to fit your needs.  The variables you will definitely want to change right away are file_date (cell #2), day_folder (cell #3), dropsonde_folder (cell #3), and lowmax_hght (cell #9).  Once you change these, everything should run properly as is.  If it doesn't, let me know.

In [1]:
import os
import sys
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
#import matplotlib.colors as mplc

import sharppy
import sharppy.sharptab.profile as profile
import sharppy.sharptab.interp as interp
import sharppy.sharptab.winds as winds
import sharppy.sharptab.utils as utils
import sharppy.sharptab.params as params
import sharppy.sharptab.thermo as thermo

# import metpy.calc as mpcalc
# import metpy.plots as mplots
# from metpy.units import units

from PIL import Image

In [2]:
file_date = '20210824'  #which date to create clean dropsonde data CSV for; only variable you need to change

In [3]:
day_folder = os.path.join(os.getcwd(), file_date)   
dropsonde_folder = os.path.join(day_folder, 'Dropsonde_files')
drop_final_name = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')

#Display the contents of a typical CPEX-AW dropsonde NetCDF file
# test = xr.open_dataset(os.path.join('/Users/brodenkirch/Desktop/CPEX/Coding/20210821/Dropsonde_files', 'CPEXAW-DROPSONDE_D20210821_221454_PQC.nc'))
# test


In [4]:
#loop through each dropsonde file to filter/QC the data and add to the given date's final_dropsonde CSV

first_file = True
for a in sorted(os.listdir(dropsonde_folder)):  #sorted() goes through the files in alphabetical order
    
    if a[-3:] != '.nc':  #grab only the dropsonde .nc files from the directory
        continue
    else:
        ds = xr.open_dataset(os.path.join(dropsonde_folder, a))
        
    #convert the dataset to a Pandas dataframe
    df = ds.to_dataframe()

    #QC the data: alt. (hydrostatic or GPS) available and > 0, pressure > 0 (and not NaN), u/v/temp/RH/dewpoint not NaN
    df_use = df[((df['alt'] > 0) | (df['gpsalt'] > 0)) & (df['pres'] > 0) & (df['u_wind'].notnull()) & 
                (df['v_wind'].notnull()) & (df['tdry'].notnull()) & (df['dp'].notnull()) & (df['rh'] >= 0)].copy()
        #^^^ using .copy() to prevent chain-indexing --> https://www.dataquest.io/blog/settingwithcopywarning/
    
    if len(df_use) != 0:
        #grab the time of the first good data line (the last index), to be used as the official dropsonde time
        drop_full_time = str(df_use.index[-1][1])[:19]

        #create one single height column, prioritizing hydrostatic altitude
        heights_use = []
        for i in range(len(df_use)):
            hydro_height = df_use['alt'][0][i]
            if hydro_height > 0:  #i.e., if the hydrostatic height value is not NaN, use hydrostatic height
                heights_use.append(hydro_height)
            else:
                heights_use.append(df_use['gpsalt'][0][i])  #if the hydrostatic height value is NaN, use GPS height
        df_use['Heights Use'] = heights_use  #needed for proper rounding of height Series....for some reason

        #convert the good, relevant dropsonde data to a dataframe and add to the final dropsonde CSV file           
        drop_clean_df = pd.DataFrame(columns = ['Time [UTC]','Height [m]', 'Pressure [mb]', 'U Comp of Wind [m/s]', 'V Comp of Wind [m/s]', 'Wind Speed [m/s]', 'Wind Direction [deg]', 'Temperature [C]', 'Dew Point [C]', 'Potential Temperature [K]', 'Relative Humidity [%]', 'Latitude [deg]', 'Longitude [deg]'])
        drop_clean_df['Time [UTC]'] = [drop_full_time] * len(df_use)  #a list of len(df_use) with the same drop_full_time value
        drop_clean_df['Height [m]'] = np.round(list(df_use['Heights Use'])[::-1], 2)
        drop_clean_df['Pressure [mb]'] = np.round(list(df_use['pres'])[::-1], 2)
        drop_clean_df['U Comp of Wind [m/s]'] = np.round(list(df_use['u_wind'])[::-1], 2)
        drop_clean_df['V Comp of Wind [m/s]'] = np.round(list(df_use['v_wind'])[::-1], 2)
        drop_clean_df['Wind Speed [m/s]'] = np.round(list(df_use['wspd'])[::-1], 2)
        drop_clean_df['Wind Direction [deg]'] = np.round(list(df_use['wdir'])[::-1], 2)
        drop_clean_df['Temperature [C]'] = np.round(list(df_use['tdry'])[::-1], 2)
        drop_clean_df['Dew Point [C]'] = np.round(list(df_use['dp'])[::-1], 2)
        drop_clean_df['Potential Temperature [K]'] = np.round(list(df_use['theta'])[::-1], 2)
        drop_clean_df['Relative Humidity [%]'] = np.round(list(df_use['rh'])[::-1], 2)
        drop_clean_df['Latitude [deg]'] = np.round(list(df_use['lat'])[::-1], 7)
        drop_clean_df['Longitude [deg]'] = np.round(list(df_use['lon'])[::-1], 7)

        if first_file:
            drop_clean_df.to_csv(drop_final_name, index = False)
            first_file = False
        else:
            df_all = pd.read_csv(drop_final_name)
            df_total = pd.concat([df_all, drop_clean_df], ignore_index = True)  #concatenates fields with same heading
            df_total.to_csv(drop_final_name, index = False)
            

In [5]:
#plot up the available, good dropsonde data for the given time range
df_all = pd.read_csv(drop_final_name)

drop_fig, drop_axs = plt.subplots(nrows=1, ncols=2, figsize = (35,20))
drop_fig.subplots_adjust(wspace=0.3)
drop_x_ax = pd.to_datetime(df_all['Time [UTC]'])
drop_y_ax = df_all['Height [m]']
drop_axs[0].scatter(drop_x_ax, drop_y_ax, s=15, c='k')
drop_axs[0].tick_params(axis='x', rotation = 50)
drop_axs[0].tick_params(labelsize=18)
drop_axs[0].grid(True)
drop_axs[0].set_xlabel('Time [UTC]', fontsize=30)
drop_axs[0].set_ylabel('Height [m]', fontsize=30)
drop_axs[0].set_title('Dropsonde Availability', fontsize=40)
drop_axs[0].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
#drop_axs[0].gcf().set_size_inches(10,13)

#plot up same figure but with wind barbs instead of dots
drop_u = df_all['U Comp of Wind [m/s]']
drop_v = df_all['V Comp of Wind [m/s]']
drop_axs[1].barbs(drop_x_ax, drop_y_ax, drop_u, drop_v, fill_empty = True, pivot='middle', sizes=dict(emptybarb=0.075), barbcolor = 'b')
#add "np.sqrt(drop_u**2 + drop_v**2)" to above line to color code barbs by speed
drop_axs[1].tick_params(axis='x', rotation = 50)
drop_axs[1].tick_params(labelsize=18)
drop_axs[1].grid(True)
drop_axs[1].set_xlabel('Time [UTC]', fontsize=30)
drop_axs[1].set_ylabel('Height [m]', fontsize=30)
drop_axs[1].set_title('Dropsonde 2-D Wind at Given Times and Heights', fontsize=40)
drop_axs[1].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
#drop_axs[1].gcf().set_size_inches(20,25)

drop_name = os.path.join(day_folder, "Dropsonde_avail_and_barbs_" + file_date + ".png")
plt.savefig(drop_name, bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
plt.clf();

<Figure size 2520x1440 with 0 Axes>

In [6]:
#make theta-e plots for each dropsonde to figure out which ones should be omitted
#exclude dropsonde profiles with large vertical data gaps and/or frequent, graphically visible anomalous spikes
    
drop_csv = pd.read_csv(drop_final_name)
drop_times = sorted(drop_csv['Time [UTC]'].unique())  #sorted() = goes through files in alphabetical order

line_types = ['b-', 'r-', 'k-', 'y-', 'b--', 'r--', 'k--', 'y--', 'c-', 'c--', 'm-', 'm--', 'b:', 'r:', 'k:', 'y:', 'c:', 'm:', 'b-.', 'r-.', 'k-.', 'y-.', 'c-.', 'm-.', 'g-', 'g--', 'g:', 'g-.']

#Potential Temperature 
fig = plt.figure(figsize=(15,15))   
    
line_index = 0
for time in drop_times:
    rel_data = drop_csv[drop_csv['Time [UTC]'] == time].copy()

    pres = rel_data['Pressure [mb]']
    hght = rel_data['Height [m]']
    tmpc = rel_data['Temperature [C]']
    dwpc = rel_data['Dew Point [C]']
    wspd = 1.94384449 * rel_data['Wind Speed [m/s]']  #converts m/s to knots (also in SHARPpy sharptab.utils script)
    wdir = rel_data['Wind Direction [deg]']

    plt.plot(rel_data['Potential Temperature [K]'], rel_data['Pressure [mb]'], line_types[line_index], label = time[11:])
    plt.xlabel("Potential Temperature [K]", fontsize = 25)
    plt.ylabel("Pressure [mb]", fontsize = 25)
    plt.ylim([1050,190])  #inverts y-axis (pressure)
    plt.yticks(np.arange(1000,190,-50))
    plt.tick_params(labelsize = 15)
    plt.legend(fontsize = 'xx-large')
    plt.grid(True)
    plt.title(file_date + ' Dropsonde Theta Profiles', fontsize = 30)
    plt.savefig(os.path.join(day_folder, 'theta_profiles.png'), bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
    line_index = line_index + 1
plt.close()
  

#Equivalent Potential Temperature
fig = plt.figure(figsize=(15,15))

line_index = 0
for time in drop_times:
    rel_data = drop_csv[drop_csv['Time [UTC]'] == time].copy()
    rel_data2 = rel_data.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

    pres = rel_data2['Pressure [mb]']
    hght = rel_data2['Height [m]']
    tmpc = rel_data2['Temperature [C]']
    dwpc = rel_data2['Dew Point [C]']
    wspd = 1.94384449 * rel_data2['Wind Speed [m/s]']  #converts m/s to knots (also in SHARPpy sharptab.utils script)
    wdir = rel_data2['Wind Direction [deg]']

    try:
        prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=True)
    except:  #if time == '2021-08-26 20:33:21'
        prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=False)
            
    plt.plot(prof.thetae.data, prof.pres.data, line_types[line_index], label = time[11:])
    plt.xlabel("Equivalent Potential Temperature [K]", fontsize = 25)
    plt.ylabel("Pressure [mb]", fontsize = 25)
    plt.ylim([1050,190])  #inverts y-axis (pressure)
    plt.yticks(np.arange(1000,190,-50))
    plt.xlim([325, 365])
    plt.xticks(np.arange(325,366,5))
    plt.tick_params(labelsize = 15)
    plt.legend(fontsize = 'xx-large')
    plt.grid(True)
    plt.title(file_date + ' Dropsonde Theta-E Profiles', fontsize = 30)
    plt.savefig(os.path.join(day_folder, 'thetaE_profiles.png'), bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
    line_index = line_index + 1
plt.close()
    
    
#Virtual Potential Temperature    
fig = plt.figure(figsize=(15,15))
    
line_index = 0
for time in drop_times:
    rel_data = drop_csv[drop_csv['Time [UTC]'] == time].copy()
    rel_data2 = rel_data.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

    pres = rel_data2['Pressure [mb]']
    hght = rel_data2['Height [m]']
    tmpc = rel_data2['Temperature [C]']
    dwpc = rel_data2['Dew Point [C]']
    wspd = 1.94384449 * rel_data2['Wind Speed [m/s]']  #converts m/s to knots (also in SHARPpy sharptab.utils script)
    wdir = rel_data2['Wind Direction [deg]']

    try:
        prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=True)
    except:  #if time == '2021-08-26 20:33:21'
        prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=False)

    thetav = thermo.theta(prof.pres.data, thermo.virtemp(prof.pres.data, prof.tmpc.data, prof.dwpc.data))   
    thetav = thermo.ctok(thetav)  #convert from Celsius to Kelvin
    plt.plot(thetav, prof.pres.data, line_types[line_index], label = time[11:])
    plt.xlabel("Virtual Potential Temperature [K]", fontsize = 25)
    plt.ylabel("Pressure [mb]", fontsize = 25)
    plt.ylim([1050,190])  #inverts y-axis (pressure)
    plt.yticks(np.arange(1000,190,-50))
    plt.tick_params(labelsize = 15)
    plt.legend(fontsize = 'xx-large')
    plt.grid(True)
    plt.title(file_date + ' Dropsonde Theta-V Profiles', fontsize = 30)
    plt.savefig(os.path.join(day_folder, 'thetaV_profiles.png'), bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
    line_index = line_index + 1
plt.close()

In [7]:
# #IF YOU HAVE CARTOPY & METPY INSTALLED, USE THIS CELL TO PLOT DROPSONDE SKEW-Ts; otherwise, use next 2 cells

# #plot dropsonde skew-T using MetPy(don't need to change anything in this cell)
# def plot_skewTs(file_date, plot_hodograph = True):
#     """ Make skew-T/hodograph figures for a given day's dropsonde profiles
    
#     PARAMETERS
#     ----------
#     file_date : the day (YYYYMMDD, string format) for which you want to plot dropsonde profile skew-Ts
#     plot_hodograph : determines whether or not to plot an inset hodograph (True/False)
    
#     RETURNS
#     ----------
#     fig : matplotlib skew-T figures for each dropsonde in the given day's "final_dropsonde_YYYYMMDD.csv" file
    
#     """
    
#     day_folder = os.path.join(os.getcwd(), file_date)   
#     dropsonde_folder = os.path.join(day_folder, 'Dropsonde_files')
#     drop_final_name = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
    
#     drop_csv = pd.read_csv(drop_final_name)
#     drop_times = drop_csv['Time [UTC]'].unique()  #sorted() = goes through files in alphabetical order
    
#     #initialize some plot visualizations
#     mpl.rcParams['font.family'] = 'arial'
#     mpl.rcParams['font.size'] = 15
#     mpl.rcParams['ytick.labelsize'] = 14
#     mpl.rcParams['xtick.labelsize'] = 14

#     for time in drop_times:

#         df0 = drop_csv[drop_csv['Time [UTC]'] == time].copy()
#         df = df0.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

#         #add appropriate units to the pressure, temperature, dewpoint, and wind data
#         pres = df['Pressure [mb]'].values * units.hPa   #hPa = mb
#         #filtered_pres = pres[::2]      #only every other pressure value for quicker profile/skew-T creation
#         temp = df['Temperature [C]'].values * units.degC
#         #filtered_temp = temp[::2]      ##only every other temp. value for quicker skew-T creation
#         dwpt = df['Dew Point [C]'].values * units.degC
#         #wnd_spd = df['Wind Speed [m/s]'].values * units('m/s')
#         wnd_spd = (df['Wind Speed [m/s]'].values * units('m/s')).to(units.knots)
#         wnd_dir = df['Wind Direction [deg]'].values * units.deg
#         #hght = df['Height [m]'].values * units.meter

#         #calculate the parcel path/profile at the near-surface for the given environment
#         #profile = mpcalc.parcel_profile(filtered_pres, temp[0], dwpt[0])  #returns temps in Kelvin
#         try:
#             profile = mpcalc.parcel_profile(pres, temp[0], dwpt[0])  #returns temps in Kelvin
#         except:
#             print (f'Could not plot {time} skew-T, likely because pressure increases between at least two points in the sounding (i.e., pressure not monotonically decreasing). Using scipy.signal.medfilt may fix this.')
#             continue
            
#         profile = profile.to('degC')

#         #calculate the LCL and wind components
#         lcl = mpcalc.lcl(pres[0], temp[0], dwpt[0])
#         wind_comps = mpcalc.wind_components(wnd_spd, wnd_dir)  #returns u,v values in whatever unit wnd_spd is in
#         u = wind_comps[0]
#         v = wind_comps[1]

#         #initialize the figure
#         fig = plt.figure(figsize = (12,12))       

#         #Initialize the skew-T figure/subplot
#         skew = mplots.SkewT(fig)

#         #Plot the data for the skew-T
#         skew.plot(pres, temp, 'darkorange', linewidth = 2)
#         skew.plot(pres, dwpt, 'cornflowerblue', linewidth = 2)
#         #skew.plot(lcl[0], lcl[1], 'yellow', marker = '*', markeredgecolor = 'k', markersize = 14)  #plot the LCL as a yellow star
#         #skew.plot(filtered_pres, profile, 'k', linewidth = 2)
#         skew.plot(pres, profile, 'k', linewidth = 2)
#         skew.plot_barbs(pres[::60], u[::60], v[::60])
#         #skew.shade_cape(filtered_pres, filtered_temp, profile)
#         #skew.shade_cin(filtered_pres, filtered_temp, profile, dwpt[::2])
#         skew.shade_cape(pres, temp, profile, alpha = 0.2)
#         skew.shade_cin(pres, temp, profile, dwpt, alpha = 0.2)
#         skew.plot_dry_adiabats(t0 = np.arange(-90, 321, 10) * units.degC, alpha = 0.3)   #range is large to cover whole plot for all possible profiles
#         skew.plot_moist_adiabats(t0 = np.arange(-90, 81, 10) * units.degC, alpha = 0.3)  #range is large to cover whole plot for all possible profiles
#         skew.ax.set_xlim(-50,40)
#         skew.ax.set_ylim(1000,200)
#         skew.ax.set_title(f'{time} Dropsonde Skew-T Diagram') #title created based on dropsonde time
#         skew.ax.set_xlabel('T [$\degree$C]')
#         skew.ax.set_ylabel('Pressure [hPa]')
        
#         #if just one sounding text file is inputted, then plot a hodograph in the upper right-hand corner of the figure
#         if plot_hodograph:
#             axh = inset_axes(skew.ax, '35%', '35%', loc = 'upper right')
#             h = mplots.Hodograph(axh, component_range = 80.)
#             h.add_grid(increment = 20)
#             h.plot_colormapped(u, v, wnd_spd);  # Plot a line colored by wind speed
        
#         save_name = os.path.join(day_folder, 'skewt_' + time[11:13] + time[14:16] + time[17:19] + '.png')
#         #plt.show()
#         plt.savefig(save_name, bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
#         plt.close()
        
#         #decrease file size of the image by 4x without noticeable image effects (if using Matplotlib)!
#         #(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
#         im = Image.open(save_name)
#         im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
#         im2.save(save_name)
#         im.close()
#         im2.close()

# plot_skewTs(file_date)

In [38]:
# #plot dropsonde skew-T using SHARPpy code (don't need to change anything in this cell)
# #it's easier to use the MetPy/cartopy cell above, 
# #so only use this if you can't download the required cartopy installation needed for metpy.plots

# from matplotlib.axes import Axes
# import matplotlib.transforms as transforms
# import matplotlib.axis as maxis
# import matplotlib.spines as mspines
# from matplotlib.projections import register_projection

# # The sole purpose of this class is to look at the upper, lower, or total
# # interval as appropriate and see what parts of the tick to draw, if any.
# class SkewXTick(maxis.XTick):
#     def update_position(self, loc):
#         # This ensures that the new value of the location is set before
#         # any other updates take place
#         self._loc = loc
#         super(SkewXTick, self).update_position(loc)

#     def _has_default_loc(self):
#         return self.get_loc() is None

#     def _need_lower(self):
#         return (self._has_default_loc() or
#                 transforms.interval_contains(self.axes.lower_xlim,
#                                               self.get_loc()))

#     def _need_upper(self):
#         return (self._has_default_loc() or
#                 transforms.interval_contains(self.axes.upper_xlim,
#                                               self.get_loc()))

#     @property
#     def gridOn(self):
#         return (self._gridOn and (self._has_default_loc() or
#                 transforms.interval_contains(self.get_view_interval(),
#                                               self.get_loc())))

#     @gridOn.setter
#     def gridOn(self, value):
#         self._gridOn = value

#     @property
#     def tick1On(self):
#         return self._tick1On and self._need_lower()

#     @tick1On.setter
#     def tick1On(self, value):
#         self._tick1On = value

#     @property
#     def label1On(self):
#         return self._label1On and self._need_lower()

#     @label1On.setter
#     def label1On(self, value):
#         self._label1On = value

#     @property
#     def tick2On(self):
#         return self._tick2On and self._need_upper()

#     @tick2On.setter
#     def tick2On(self, value):
#         self._tick2On = value

#     @property
#     def label2On(self):
#         return self._label2On and self._need_upper()

#     @label2On.setter
#     def label2On(self, value):
#         self._label2On = value

#     def get_view_interval(self):
#         return self.axes.xaxis.get_view_interval()


# # This class exists to provide two separate sets of intervals to the tick,
# # as well as create instances of the custom tick
# class SkewXAxis(maxis.XAxis):
#     def _get_tick(self, major):
#         return SkewXTick(self.axes, None, '', major=major)

#     def get_view_interval(self):
#         return self.axes.upper_xlim[0], self.axes.lower_xlim[1]


# # This class exists to calculate the separate data range of the
# # upper X-axis and draw the spine there. It also provides this range
# # to the X-axis artist for ticking and gridlines
# class SkewSpine(mspines.Spine):
#     def _adjust_location(self):
#         pts = self._path.vertices
#         if self.spine_type == 'top':
#             pts[:, 0] = self.axes.upper_xlim
#         else:
#             pts[:, 0] = self.axes.lower_xlim


# # This class handles registration of the skew-xaxes as a projection as well
# # as setting up the appropriate transformations. It also overrides standard
# # spines and axes instances as appropriate.
# class SkewXAxes(Axes):
#     # The projection must specify a name.  This will be used be the
#     # user to select the projection, i.e. ``subplot(111,
#     # projection='skewx')``.
#     name = 'skewx'

#     def _init_axis(self):
#         # Taken from Axes and modified to use our modified X-axis
#         self.xaxis = SkewXAxis(self)
#         self.spines['top'].register_axis(self.xaxis)
#         self.spines['bottom'].register_axis(self.xaxis)
#         self.yaxis = maxis.YAxis(self)
#         self.spines['left'].register_axis(self.yaxis)
#         self.spines['right'].register_axis(self.yaxis)

#     def _gen_axes_spines(self):
#         spines = {'top': SkewSpine.linear_spine(self, 'top'),
#                   'bottom': mspines.Spine.linear_spine(self, 'bottom'),
#                   'left': mspines.Spine.linear_spine(self, 'left'),
#                   'right': mspines.Spine.linear_spine(self, 'right')}
#         return spines

#     def _set_lim_and_transforms(self):
#         """
#         This is called once when the plot is created to set up all the
#         transforms for the data, text and grids.
#         """
#         rot = 30

#         # Get the standard transform setup from the Axes base class
#         Axes._set_lim_and_transforms(self)

#         # Need to put the skew in the middle, after the scale and limits,
#         # but before the transAxes. This way, the skew is done in Axes
#         # coordinates thus performing the transform around the proper origin
#         # We keep the pre-transAxes transform around for other users, like the
#         # spines for finding bounds
#         self.transDataToAxes = self.transScale + \
#             self.transLimits + transforms.Affine2D().skew_deg(rot, 0)

#         # Create the full transform from Data to Pixels
#         self.transData = self.transDataToAxes + self.transAxes

#         # Blended transforms like this need to have the skewing applied using
#         # both axes, in axes coords like before.
#         self._xaxis_transform = (transforms.blended_transform_factory(
#             self.transScale + self.transLimits,
#             transforms.IdentityTransform()) +
#             transforms.Affine2D().skew_deg(rot, 0)) + self.transAxes

#     @property
#     def lower_xlim(self):
#         return self.axes.viewLim.intervalx

#     @property
#     def upper_xlim(self):
#         pts = [[0., 1.], [1., 1.]]
#         return self.transDataToAxes.inverted().transform(pts)[:, 0]
    

In [9]:
# #plot dropsonde skew-T using SHARPpy code (continued...)
# #it's easier to use the MetPy/cartopy cell above, 
# #so only use this if you can't download the required cartopy installation needed for metpy.plots

# drop_csv = pd.read_csv(drop_final_name)
# drop_times = drop_csv['Time [UTC]'].unique()  #sorted() = goes through files in alphabetical order

# for time in drop_times:
    
#     rel_data = drop_csv[drop_csv['Time [UTC]'] == time].copy()
#     rel_data2 = rel_data.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

#     pres = rel_data2['Pressure [mb]']
#     hght = rel_data2['Height [m]']
#     tmpc = rel_data2['Temperature [C]']
#     dwpc = rel_data2['Dew Point [C]']
#     wspd = 1.94384449 * rel_data2['Wind Speed [m/s]']  #converts m/s to knots (also in SHARPpy sharptab.utils script)
#     wdir = rel_data2['Wind Direction [deg]']

#     try:
#         prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=True)
#     except:  #if time == '2021-08-26 20:33:21'
#         prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=False)
    
#     # Select the Most-Unstable parcel (this can be changed)
#     mupcl = params.parcelx(prof, flag=3, exact = True) # Most-Unstable Parcel
#     pcl = mupcl
    
#     # Now register the projection with matplotlib so the user can select it.
#     register_projection(SkewXAxes)

#     from matplotlib.ticker import (MultipleLocator, NullFormatter, ScalarFormatter)

#     # Create a new figure. The dimensions here give a good aspect ratio
#     fig = plt.figure(figsize=(6.5875, 6.2125))
#     ax = fig.add_subplot(111, projection='skewx')
#     ax.grid(True)

#     # Let's set the y-axis bounds of the plot.
#     pmax = 1000
#     pmin = 10
#     dp = -10
#     presvals = np.arange(int(pmax), int(pmin)+dp, dp)

#     # plot the moist-adiabats at surface temperatures -10 C to 45 C at 5 degree intervals.
#     for t in np.arange(-10,45,5):
#         tw = []
#         for p in presvals:
#             tw.append(thermo.wetlift(1000., t, p))
#         # Plot the moist-adiabat with a black line that is faded a bit.
#         ax.semilogy(tw, presvals, 'k-', alpha=.2)

#     # A function to calculate the dry adiabats
#     def thetas(theta, presvals):
#         return ((theta + thermo.ZEROCNK) / (np.power((1000. / presvals),thermo.ROCP))) - thermo.ZEROCNK

#     # plot the dry adiabats
#     for t in np.arange(-50,110,10):
#         ax.semilogy(thetas(t, presvals), presvals, 'r-', alpha=.2)

#     # plot the title.
#     plt.title(file_date + ' ' + time[11:] + 'Z (Observed)', fontsize=14, loc='left')

#     # Plot the data using normal plotting functions, in this case using
#     # log scaling in Y, as dicatated by the typical meteorological plot
#     ax.semilogy(prof.tmpc.data, prof.pres.data, 'r', lw=2)
#     ax.semilogy(prof.dwpc.data, prof.pres.data, 'g', lw=2)

#     # Plot the parcel trace.
#     ax.semilogy(pcl.ttrace, pcl.ptrace, 'k-.', lw=2)

#     # Denote the 0 to -20 C area on the Skew-T.
#     l = ax.axvline(0, color='b', linestyle='--')
#     l = ax.axvline(-20, color='b', linestyle='--')

#     # Set the log-scale formatting and label the y-axis tick marks.
#     ax.yaxis.set_major_formatter(plt.ScalarFormatter())
#     ax.set_yticks(np.linspace(100,1000,10))
#     ax.set_ylim(1050,100)

#     # Label the x-axis tick marks.
#     ax.xaxis.set_major_locator(plt.MultipleLocator(10))
#     ax.set_xlim(-50,50)

#     # Show the plot to the user.
#     skewt_path = os.path.join(day_folder, 'skewt_' + time[11:13] + time[14:16] + time[17:19] + '.png')
#     plt.savefig(skewt_path, bbox_inches='tight') # saves the plot to the disk.
#     # plt.show()
#     plt.close()

In [41]:
#Calculate mean-layer metrics for each dropsonde using SHARPpy
#metrics will be copy/pasted into Dropsonde_Metric_Calculations.csv
lowmax_hght = 7622.5  #lowest max dropsonde height (meters) of all dropsondes in Dropsonde_Metric_Calculations.csv (used to use 8595.5 m)
drop_csv = pd.read_csv(drop_final_name)
drop_times = drop_csv['Time [UTC]'].unique()  #sorted() = goes through files in alphabetical order

for time in drop_times:
#     print (time)
#     if time == '2017-06-11 19:05:20':  #this dropsonde does not reach even close to the freezing level, which causes problems
#         continue
        
    rel_data = drop_csv[drop_csv['Time [UTC]'] == time].copy()
    rel_data2 = rel_data.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

    pres = rel_data2['Pressure [mb]']
    hght = rel_data2['Height [m]']
    tmpc = rel_data2['Temperature [C]']
    dwpc = rel_data2['Dew Point [C]']
    wspd = 1.94384449 * rel_data2['Wind Speed [m/s]']  #converts m/s to knots (also in SHARPpy sharptab.utils script)
    wdir = rel_data2['Wind Direction [deg]']
    
    list_pres = pres.tolist()   
    list_hght = hght.tolist()
    list_wspd = wspd.tolist()
    list_wdir = wdir.tolist()
    
    try:
        prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=True)
    except:  #if time == '2021-08-26 20:33:21'
        prof = profile.create_profile(profile='default', pres=pres, hght=hght, tmpc=tmpc, dwpc=dwpc, wspd=wspd, wdir=wdir, missing=-9999, strictQC=False)    
    
    sfc_p = prof.pres.data[prof.sfc]
    sfc_hght = prof.hght.data[prof.sfc]
    
    freeze_lev = params.temp_lvl(prof, temp = 0)
#    print ('Profile Pressure Value at 0 C: %.1f' % freeze_lev)
    m10c_lev = params.temp_lvl(prof, temp = -10)
#    print ('Profile Pressure Value at -10 C: %.1f' % m10c_lev)
    m20c_lev = params.temp_lvl(prof, temp = -20)
#    print ('Profile Pressure Value at -20 C: %.1f' % m20c_lev)
    m30c_lev = params.temp_lvl(prof, temp = -30)
#    print ('Profile Pressure Value at -30 C: %.1f' % m30c_lev)
    
    # Find the pressure/height value that corresponds closest to the lowest max dropsonde height without going over
    x = 0
    upper_lvl_hcap = -999.0  #just in case the dropsonde data doesn't reach the lowmax_hght when falling, this will alert you to skip the dropsonde
    while prof.hght.data[x] <= lowmax_hght:
        upper_lvl_pcap = prof.pres.data[x]
        upper_lvl_hcap = prof.hght.data[x]
        if x == (len(prof.hght.data) - 1):       #stops the loop if the highest (lowest) height (pressure) in the profile was just reached/assigned
            break
        else:
            x += 1
    #x = np.argmin(abs(prof.hght.data - lowmax_hght))
    #upper_lvl_pcap = prof.pres.data[x]
    #upper_lvl_hcap = prof.hght.data[x]
    if upper_lvl_hcap < 0:  #i.e., if the dropsonde data doesn't reach the lowmax_hght when falling
        print ('Bad dropsonde: Dropsonde at ', time, ' did not reach the lowmax_hght. \n')
        continue
    
    # Calculate PBL Top using the virtual potential temperature method (see Ajda's work and params.pbl_top code)
    pbl_top_pres = params.pbl_top(prof)
#    print ('PBL Top: %.1f mb' % pbl_top_pres) # mb

    PBL_bot_index = 0  #i.e. list_pres.index(sfc_p)
    PBL_top_index = list_pres.index(pbl_top_pres)
    pbl_top_hght = prof.hght.data[PBL_top_index]
    
    mid_bot_index = PBL_top_index + 1
    upper_top_index = list_pres.index(upper_lvl_pcap)
    
    #Since freeze_lev isn't always a literal pressure entry in the profile, need to find the
        #lowest pressure in the mid layer (i.e., the pressure level closest to the freezing level without going over)
    z = 0
    freeze_lev_hght = -999.0  #just in case the dropsonde data doesn't reach the freezing level when falling, this will alert you to skip the dropsonde
    while prof.pres.data[z] >= freeze_lev:
        mid_top_index = z
        freeze_lev_hght = prof.hght.data[mid_top_index]
        z += 1
    upper_bot_index = z  #i.e. mid_top_index + 1
    #z = np.argmin(abs(prof.pres.data - freeze_lev))
    #mid_top_index = prof.pres.data[z]
    #upper_bot_index = prof.pres.data[z+1]
    if freeze_lev_hght < 0:  #i.e., if the dropsonde data doesn't reach the freezing level when falling
        print ('Bad dropsonde: Dropsonde at ', time, ' did not reach the freezing level. \n')
        continue

    print ('Stats for Dropsonde at:', time, '\n')
#    print ('Dropsonde max height: %.1f m' % prof.hght.data[-1])
#    print ('Surface Pressure: %.1f mb' % sfc_p)
#    print ('Upper-level Pressure Cap: %.1f mb' % upper_lvl_pcap)
    
    #mean-layer RH calculations based off of SHARPpy pressure threshold values (i.e. sfc - PBL Top, PBL Top - freezing level, freezing level - upper_lvl_pcap)
    PBL_RH_average = rel_data2[rel_data2['Pressure [mb]'] >= pbl_top_pres]['Relative Humidity [%]'].mean(skipna = True)
    mid_RH_average = rel_data2[(rel_data2['Pressure [mb]'] < pbl_top_pres) & (rel_data2['Pressure [mb]'] >= freeze_lev)]['Relative Humidity [%]'].mean(skipna = True)
    upper_RH_average = rel_data2[(rel_data2['Pressure [mb]'] < freeze_lev) & (rel_data2['Pressure [mb]'] >= upper_lvl_pcap)]['Relative Humidity [%]'].mean(skipna = True)
    deep_RH_average = rel_data2[rel_data2['Pressure [mb]'] >= upper_lvl_pcap]['Relative Humidity [%]'].mean(skipna = True)
    
#     print ('PBL RH %:  ', np.round(PBL_RH_average,1))
#     print ('Mid Layer RH %:  ', np.round(mid_RH_average,1))  
#     print ('Upper Layer RH %:  ', np.round(upper_RH_average,1))   
#     print ('Deep Layer RH %:  ', np.round(deep_RH_average,1))
    
    mupcl = params.parcelx(prof, flag=3, exact = True) # Most-Unstable Parcel
    mlpcl = params.parcelx(prof, flag=4, exact = True) # 100 mb Mean Layer Parcel (from sfc. to sfc. - 100mb)
    Dmupcl = params.parcelx(prof, flag=3, ptop = upper_lvl_pcap, exact = True) # Deep Layer Most-Unstable Parcel
    Bmupcl = params.parcelx(prof, flag=3, ptop = freeze_lev, exact = True) # Below Freezing Level Most-Unstable Parcel
    Dmlpcl = params.parcelx(prof, flag=4, ptop = upper_lvl_pcap, exact = True) # Deep layer 100 mb Mean Layer Parcel (from sfc. to sfc. - 100mb)
    Bmlpcl = params.parcelx(prof, flag=4, ptop = freeze_lev, exact = True) # Below Freezing Level 100 mb Mean Layer Parcel (from sfc. to sfc. - 100mb)    
    
    below_mucape = Bmupcl.bplus                #Below Freezing Level MUCAPE
    deep_mucape = Dmupcl.bplus                 #Deep Layer MUCAPE
    above_mucape = deep_mucape - below_mucape  #Above Freezing Level MUCAPE
    below_mlcape = Bmlpcl.bplus                #Below Freezing Level MLCAPE
    deep_mlcape = Dmlpcl.bplus                 #Deep Layer MLCAPE
    above_mlcape = deep_mlcape - below_mlcape  #Above Freezing Level MLCAPE

    print ('Paste the following into Dropsonde_Metric_Calculations.csv:')
    print (' ')
    print (np.round(sfc_p, 1))            #Metrics independent of parcel type start here
    print (np.round(sfc_hght, 1))
    print (np.round(pbl_top_pres, 1))
    print (np.round(pbl_top_hght, 1))
    print (np.round(freeze_lev, 1))
    print (np.round(freeze_lev_hght, 1))
    print (np.round(upper_lvl_pcap, 1))
    print (np.round(upper_lvl_hcap, 1))
    print (np.round(m10c_lev, 1))
    print (np.round(m20c_lev, 1))
    print (np.round(m30c_lev, 1))
    print (np.round(PBL_RH_average, 1))
    print (np.round(mid_RH_average, 1))
    print (np.round(upper_RH_average, 1))   
    print (np.round(deep_RH_average, 1))
    print (np.round(below_mucape, 1))     #MU metrics start here
    print (np.round(above_mucape, 1))
    print (np.round(deep_mucape, 1))
    print (np.round(Dmupcl.bminus, 1))
    print (np.round(Dmupcl.lclpres, 1))
    print (np.round(Dmupcl.elpres, 1))
    print (np.round(Dmupcl.limax, 2))
    print (np.round(Dmupcl.limaxpres, 1))
    print ('')  #MU Max LI Layer (inputted manually)
    print (np.round(below_mlcape, 1))     #ML metrics start here
    print (np.round(above_mlcape, 1))
    print (np.round(deep_mlcape, 1))
    print (np.round(Dmlpcl.bminus, 1))
    print (np.round(Dmlpcl.lclpres, 1))
    print (np.round(Dmlpcl.elpres, 1))
    print (np.round(Dmlpcl.limax, 2))
    print (np.round(Dmlpcl.limaxpres, 1))
    print ('')  #ML Max LI Layer (inputted manually if you want, though not used in analysis)
    
    #SHARPpy direct shear calculation method
    PBL_shear = winds.wind_shear(prof, pbot = sfc_p, ptop = pbl_top_pres)
    PBL_speed3 = utils.comp2vec(PBL_shear[0], PBL_shear[1])[1]
    PBL_dir3 = utils.comp2vec(PBL_shear[0], PBL_shear[1])[0]
    mid_shear = winds.wind_shear(prof, pbot = list_pres[mid_bot_index], ptop = list_pres[mid_top_index])
    mid_speed3 = utils.comp2vec(mid_shear[0], mid_shear[1])[1]
    mid_dir3 = utils.comp2vec(mid_shear[0], mid_shear[1])[0]
    upper_shear = winds.wind_shear(prof, pbot = list_pres[upper_bot_index], ptop = upper_lvl_pcap)
    upper_speed3 = utils.comp2vec(upper_shear[0], upper_shear[1])[1]
    upper_dir3 = utils.comp2vec(upper_shear[0], upper_shear[1])[0]
    deep_shear = winds.wind_shear(prof, pbot = sfc_p, ptop = upper_lvl_pcap)
    deep_speed3 = utils.comp2vec(deep_shear[0], deep_shear[1])[1]
    deep_dir3 = utils.comp2vec(deep_shear[0], deep_shear[1])[0]
    
    print (np.round(PBL_speed3, 2))
    print (np.round(PBL_dir3, 1))
    print (np.round(mid_speed3, 2))
    print (np.round(mid_dir3, 1))
    print (np.round(upper_speed3, 2))
    print (np.round(upper_dir3, 1))
    print (np.round(deep_speed3, 2))
    print (np.round(deep_dir3, 1))  
    
    #relevant theta-gradient mean-layer RH values (for PBL and mid-layer) (not really relevant anymore)
    print ('')  #10mb Interval Theta Gradient PBL Top [mb], if calculated (see download_files.py)
    print ('')  #10mb Interval Theta Gradient PBL RH [%], if calculated (see download_files.py)
    print ('')  #10mb Interval Theta Gradient Mid Layer RH [%], if calculated (see download_files.py)
    print (prof.hght.data[-1])   #dropsonde max height for records
    
    #500m Bottom Cap Deep Layer Shear
    index500 = np.argmin(abs(prof.hght.data - 500))
    p500 = prof.pres.data[index500]
    h500 = prof.hght.data[index500]
    deep_500m_shear = winds.wind_shear(prof, pbot = p500, ptop = upper_lvl_pcap)
    deep_500m_dir, deep_500m_speed = utils.comp2vec(deep_500m_shear[0], deep_500m_shear[1])
    
    print (np.round(h500, 1))  #dropsonde height closest to 500m (used for 500m - upper_lvl_cap deep shear calculation)
    print (np.round(deep_500m_speed, 2))
    print (np.round(deep_500m_dir, 1))
    print (' ')
    

Bad dropsonde: Dropsonde at  2017-06-16 19:47:18  did not reach the freezing level. 

Stats for Dropsonde at: 2017-06-16 18:52:41 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)


Paste the following into Dropsonde_Metric_Calculations.csv:
 
801.2
2018.5
771.7
2335.0
560.1
4976.9
400.0
7621.0
438.3
354.3
293.2
43.0
83.4
78.5
80.6
50.7
24.3
75.0
-7.0
717.7
504.9
-1.64
583.7

0.0
0.0
0.0
0.0
672.3
672.3
3.55
671.8

8.32
259.1
18.47
271.2
7.86
171.4
27.31
250.0



10024.9
2018.5
27.31
250.0
 
Stats for Dropsonde at: 2017-06-16 21:04:40 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)


Paste the following into Dropsonde_Metric_Calculations.csv:
 
861.2
1389.2
853.1
1470.3
567.8
4841.2
399.5
7611.8
433.4
352.4
291.1
89.5
66.8
82.3
76.7
46.5
1.4
47.9
-21.4
813.5
553.9
-1.99
603.9

23.9
0.0
23.9
-54.9
815.0
566.7
-1.6
603.9

1.39
213.4
25.14
283.0
13.33
156.8
21.79
248.0



12279.2
1389.2
21.79
248.0
 
Stats for Dropsonde at: 2017-06-16 21:08:04 



...

Paste the following into Dropsonde_Metric_Calculations.csv:
 
1002.0
103.4
931.6
745.8
552.2
5133.6
403.2
7609.9
441.5
359.9
295.3
84.5
77.2
71.5
76.2
214.7
333.1
547.8
0.0
946.8
--
--
--

47.7
163.7
211.4
-32.3
914.8
--
-0.06
657.0

3.14
330.4
7.98
180.1
6.71
247.8
12.15
230.2



9991.4
496.2
13.07
226.8
 
Stats for Dropsonde at: 2017-06-16 18:50:14 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)


Paste the following into Dropsonde_Metric_Calculations.csv:
 
996.7
146.9
980.9
287.7
565.8
4904.9
400.6
7621.8
439.4
349.9
294.3
83.5
85.0
86.4
85.5
158.5
86.9
245.4
-31.5
955.8
397.8
-2.57
631.4

70.5
11.3
81.9
-34.0
902.9
491.9
-1.72
631.4

2.5
128.7
14.44
226.6
7.42
312.9
15.39
245.4



9991.6
504.4
17.76
252.1
 
Stats for Dropsonde at: 2017-06-16 19:08:29 



/Users/brodenkirch/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:1423: UserWarning: Warning: converting a masked element to nan.
  return interp_func(x, xp, fp, left, right)


Paste the following into Dropsonde_Metric_Calculations.csv:
 
812.2
1911.2
806.8
1968.0
550.4
5128.0
400.9
7613.3
441.0
362.7
290.7
79.8
83.3
80.7
82.1
0.0
1.3
1.3
-24.3
591.5
385.1
-0.2
410.3

0.0
0.0
0.0
0.0
723.2
723.2
2.06
722.8

2.46
198.3
8.54
17.3
11.9
231.7
7.58
256.6



9986.1
1911.2
7.58
256.6
 
